# Migration from Wave App to Toshl
This notebook to migrate my accounting data from WaveApp (CSV) to Toshl (API).

In [ ]:
# import the required modules
import requests, json
import re
import pandas as pd
import numpy as np

In [ ]:
# enter your tokens in config.json.example and rename to config.json
with open('config.json', 'r') as f:
    config = json.load(f)

wave_token = config['CONFIG']['WAVEAPP_TOKEN']
toshl_token = config['CONFIG']['TOSHL_TOKEN']

# replace the id with yours. You can find it in the URL when connected to WaveApp web.
# This is useful only with you have mutli currency as the CSV only gives you the main currency.
wave_web_account_id = config['CONFIG']['WAVEAPP_WEB_ACCOUNT_ID']

## Connection to Wave App API
The get transactions method isn't available in the API yet. So we will use the CSV instead.
The code below shows how to connect to the GraphQL API from WaveApp.

In [ ]:
!pip install gql

In [ ]:
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport

In [ ]:
sample_transport = RequestsHTTPTransport(
    url = 'https://gql.waveapps.com/graphql/public',
    use_json = True,
    headers = {'Content-Type': 'application/json',
               'Authorization': 'Bearer {}'.format(wave_token)},
    verify = True
)

client = Client(
    retries = 3,
    transport = sample_transport,
    fetch_schema_from_transport = True,
)

In [ ]:
query = gql('''

    query {
      businesses(page: 1, pageSize: 10) {
        pageInfo {
          currentPage
          totalPages
          totalCount
        }
        edges {
          node {
            id
            name
            isPersonal
          }
        }
      }
    }
    
''')

In [ ]:
data = client.execute(query)

data

# Access token for transactions

I've reverse engineered the API by checking what endpoints are invoked on the website.

In [ ]:
def get_wave_web_data(account_guid, order = 'DESC'):

    url = 'https://api.waveapps.com'
    url += '/businesses/{}/transactions/latest/?account_guid={}'.format(wave_web_account_id, account_guid)
    url += '&page_size=200&show_hidden_transactions=false&sort_order={}&include_full_transaction=false'.format(order)

    headers = {'Content-Type': 'application/vnd.toshl+json',
               'Authorization': 'Bearer {}'.format(wave_token)}

    response = requests.get(url, headers=headers)
    data = response.json()
    df_waveapps_web = pd.DataFrame(data['latest_transactions'])
    
    return df_waveapps_web

# Some data cleaning

In [ ]:
def waveapps_web_cleaning(df):
    df = df.drop(['is_credit_card_payment',
                  'origin',
                  'sequence',
                  'tax_account_names',
                  'transaction',
                  'date_created',
                  'sort_key',
                  'transaction_guid',
                  'transaction_status',
                  'transaction_type',
                  'quick_categorization',
                  'direction',
                  'line_item_account_names'], axis=1)
    
    df['account_value'] = ''
    df['account_currency'] = ''
    df['transaction_value'] = ''
    df['transaction_currency'] = ''

    for i in df.index:
        df['account_value'].iloc[i] = df.transaction_total.iloc[i]['business']['value']
        df['account_currency'].iloc[i] = df.transaction_total.iloc[i]['business']['currency']['code']
        df['transaction_value'].iloc[i] = df.transaction_total.iloc[i]['transaction']['value']
        df['transaction_currency'].iloc[i] = df.transaction_total.iloc[i]['transaction']['currency']['code']

    df = df.drop(['transaction_total'], axis=1)
    
    return df

## Toshl API

In [ ]:
from requests.exceptions import HTTPError

def get_toshl(endpoint, api_token = toshl_token):
    api_url_base = 'https://api.toshl.com'
    api_url = api_url_base + endpoint
    headers = {'Content-Type': 'application/vnd.toshl+json',
               'Authorization': 'Bearer {}'.format(api_token)}
    
    response = requests.get(api_url, headers=headers)
    data = response.json()
        
    return data


def post_toshl(endpoint, post_data = {}, api_token = toshl_token):
    api_url_base = 'https://api.toshl.com'
    api_url = api_url_base + endpoint
    headers = {'Content-Type': 'application/vnd.toshl+json',
               'Authorization': 'Bearer {}'.format(api_token)}
    
    try:
        response = requests.post(api_url, headers=headers, json = post_data)

        # If the response was successful, no Exception will be raised
        response.raise_for_status()
        return response.headers['Location']
    except HTTPError as http_err:
        print(f'\nHTTP error occurred: {http_err}\n')
        print(f'Data:\n\n{post_data}\n')
        print(response.json())
        return response.json()
    except Exception as err:
        print(f'Other error occurred: {err}\n')
        return response.json()
    else:
        print('Success!\n')
        return response.headers


def detele_toshl(endpoint, api_token = toshl_token):
    api_url_base = 'https://api.toshl.com'
    api_url = api_url_base + endpoint
    headers = {'Content-Type': 'application/vnd.toshl+json',
               'Authorization': 'Bearer {}'.format(api_token)}
    
    response = requests.delete(api_url, headers=headers)
    data = response.status_code
        
    return data


def delete_all_toshl(account):
    try:
        df_toshl = pd.DataFrame(get_toshl(
            '/entries?from=2018-01-01&to=2050-12-31&accounts={}&per_page=400'.format(account)))
        for trans_id in df_toshl.id:
            endpoint = '/entries/{}'.format(trans_id)
            detele_toshl(endpoint)
        print('Success.')
    except AttributeError as attr_err:
        print('The account is empty.')

# Get accounts, categories, tags

In [ ]:
toshl_accounts = pd.DataFrame(get_toshl('/accounts'))
toshl_categories = pd.DataFrame(get_toshl('/categories'))
toshl_tags = pd.DataFrame(get_toshl('/tags?per_page=200'))

def get_account_id(name):
    account_id = toshl_accounts.loc[toshl_accounts['name'] == name]['id'].values
    if account_id.size == 0:
        return 'No account with this name.'
    else:
        return account_id[0]
    
def get_category_id(name, cat_type):
    category_id = toshl_categories.loc[(toshl_categories['name'] == name) & (toshl_categories['type'] == cat_type)]['id'].values
    if category_id.size == 0:
        return 'No category with this name.'
    else:
        return category_id[0]
    
def get_tag_id(name, tag_type):
    tag_id = toshl_tags.loc[(toshl_tags['name'] == name) & (toshl_tags['type'] == tag_type)]['id'].values
    if tag_id.size == 0:
        return 'No tag with this name.'
    else:
        return tag_id[0]
    
def get_tags_by_category(category_id):
    tags = toshl_tags.loc[toshl_tags['category'] == str(category_id)]['name'].values
    if tags.size == 0:
        return 'No tags with in this category.'
    else:
        return tags

In [ ]:
toshl_accounts.head(3)

In [ ]:
toshl_categories.head()

In [ ]:
toshl_tags.head()

# Import and clean CSV from Wave

In [ ]:
wave_accounting = pd.read_csv('wave-accounting.csv')
wave_accounting.head()

In [ ]:
# drop empty column
wave_accounting_clean = wave_accounting.drop(wave_accounting.columns[6], axis=1)

wave_accounting_clean = wave_accounting_clean.drop(['Transaction Line Description',
                                                    'Customer',
                                                    'Vendor',
                                                    'Invoice Number',
                                                    'Bill Number',
                                                    'Debit Amount (Two Column Approach)',
                                                    'Credit Amount (Two Column Approach)',
                                                    'Sales Tax Amount',
                                                    'Sales Tax Name',
                                                    'Amount Before Sales Tax',
                                                    'Transaction Date Added',
                                                    'Transaction Date Last Modified',
                                                    'Account ID'], axis=1)

In [ ]:
# concat the description in one column
for i in wave_accounting_clean.index:
    if isinstance((wave_accounting_clean.loc[i, 'Notes / Memo']), type(str)):
        description = wave_accounting_clean.loc[i, 'Transaction Description'] + ' / '  \
                    + wave_accounting_clean.loc[i, 'Notes / Memo']

    else:
        description = wave_accounting_clean.loc[i, 'Transaction Description']

    wave_accounting_clean.loc[i, 'desc'] = description

In [ ]:
wave_accounting_clean = wave_accounting_clean.drop(['Notes / Memo',
                                                    'Account Type'], axis=1)
wave_accounting_clean = wave_accounting_clean.rename(columns = {'Other Accounts for this Transaction':'category'})

In [ ]:
wave_accounting_clean.head()

In [ ]:
print('Before cleaning: {} rows and {} columns'.format(wave_accounting_clean.shape[0],
                                                       wave_accounting_clean.shape[1]))

# name of the DataFrame
df = pd.DataFrame(wave_accounting_clean)

get_trans_id = lambda i : df.iloc[i].at['Transaction ID']
get_trans_amt = lambda i : abs(df.iloc[i].at['Amount (One column)'])
def count_transaction_id(index):
    return len(df[df['Transaction ID'] == get_trans_id(index)])

# create a new column
df['split'] = ''

# adapt the amount for credit cards
for i in df.index:
    if df['Account Group'].loc[i] == 'Liability':
        df.loc[i, 'Amount (One column)'] = np.negative(df.loc[i, 'Amount (One column)'])

i = 0
while i < df.shape[0]-1:
    # try if index exists
    try:
        # if same transaction and same amount
        if get_trans_id(i) == get_trans_id(i+1) and get_trans_amt(i) == get_trans_amt(i+1):
            df = df.drop([df.iloc[i+1].name])
            i += 1

        # if split transaction
        elif count_transaction_id(i) > 2:
            # change category name
            df.loc[df.iloc[i].name, 'category'] = 'Split'
            df.iat[i, -1] = {}
            ii = i + 1
            while get_trans_id(i) == get_trans_id(ii):
                # deal with sub transactions
                # if credit card transaction
                if df.iloc[i].at['Account Group'] == 'Liability':
                    df.iat[i, -1][df.iloc[ii].at['Account Name']] = np.negative(df.iloc[ii].at['Amount (One column)'])
                else:
                    df.iat[i, -1][df.iloc[ii].at['Account Name']] = df.iloc[ii].at['Amount (One column)']
                        
                # adapt correct amount
                if float(df.iloc[i].at['Amount (One column)']) < 0:
                    df.iat[i, -1][df.iloc[ii].at['Account Name']]= np.negative(
                        abs(float(df.iloc[ii].at['Amount (One column)'])))
                else:
                    df.iat[i, -1][df.iloc[ii].at['Account Name']] = abs(float(df.iloc[ii].at['Amount (One column)']))
            
                df = df.drop([df.iloc[ii].name])

        # check if transfer clearning 
        elif get_trans_id(i) == get_trans_id(i+1) \
        and get_trans_amt(i) != get_trans_amt(i+1) \
        and count_transaction_id(i) == 2 \
        and df.iloc[i+1].at['category'] == 'Transfer Clearing':
            df = df.drop([df.iloc[i+1].name])
            i += 1

        else:
            i += 1
            
    except IndexError:
        break
        
        
print('After cleaning: {} rows and {} columns'.format(df.shape[0], df.shape[1]))
            
df.head()

In [ ]:
%config InlineBackend.figure_format = 'retina'
print(df['Account Name'].value_counts())

In [ ]:
df['Account Name'].value_counts().plot.bar(rot=45);

In [ ]:
df.loc[df['Account Name'] == 'Transfer Clearing']

In [ ]:
ids = df["Transaction ID"]
df[ids.isin(ids[ids.duplicated()])]

# Map new category

Choose the categories that fit the best.

In [ ]:
# Choose either income or expense
category_tag_type = 'income'

print('{} categories:\n'.format(category_tag_type))
for category in toshl_categories[toshl_categories['type'] == category_tag_type]['name'].values:
    print(category)

In [ ]:
# Change with the category you want to see the tags
category = 'Salary'

for tag in get_tags_by_category(get_category_id(category, category_tag_type)):
    print(tag)

In [ ]:
categories = {'Restaurants, Coffee & Bars': {'category' : 'Food & Drinks',
                                             'tags': ['bar', 'coffee & tea', 'restaurants']},
            'Groceries': {'category' : 'Food & Drinks',
                          'tags': ['groceries']},
            'Travel & Vacation': {'category' : 'Leisure',
                                  'tags': ['travel']},
            'Rental Car & Taxi': {'category' : 'Transport',
                                  'tags': ['rental', 'taxi']},
            'Gym, Sports & Recreation': {'category' : 'Sports',
                                         'tags': []},
            'Vehicle – Fuel': {'category' : 'Transport',
                               'tags': ['fuel']},
            'Alcohol & Tobacco': {'category' : 'Food & Drinks',
                                  'tags': ['alcohol']},
            'Public Transportation': {'category' : 'Transport',
                                      'tags': ['public transport']},
            'Grooming & Beauty': {'category' : 'Health & Personal Care',
                                  'tags': []},
            'Accounts Payable': {'category' : 'Loans',
                                 'tags': ['payable']},
            'Mortgage or Rent': {'category' : 'Home & Utilities',
                                 'tags': ['rent']},
            'Pharmacy & Prescriptions': {'category' : 'Health & Personal Care',
                                         'tags': ['medicine']},
            'Entertainment': {'category' : 'Leisure',
                              'tags': []},
            'Cleaning': {'category' : 'Home & Utilities',
                         'tags': []},
            'Mobile Phone': {'category' : 'Home & Utilities',
                             'tags': ['mobile phone']},
            'Other Transportation': {'category' : 'Transport',
                                     'tags': []},
            'Utilities': {'category' : 'Home & Utilities',
                          'tags': []},
            'Owner Investment / Drawings': {'category' : 'Loans',
                                            'tags': ['owner investment']},
            'Vehicle – Repairs & Maintenance': {'category' : 'Transport',
                                                'tags': ['servicing']},
            'Transfer Clearing': {'category' : 'Transfer',
                                  'tags': []},
            'Bank & Credit Card Fees': {'category' : 'Other',
                                        'tags': ['bank fees']},
            'Personal Items': {'category' : 'Clothing & Footwear',
                               'tags': []},
            'Accounts Receivable': {'category' : 'Loans',
                                    'tags': ['receivable']},
            'Split': {'category' : 'Other',
                      'tags': []},
            'Doctor, Dentist, etc.': {'category' : 'Health & Personal Care',
                                      'tags': ['medical services']},
            'Internet': {'category' : 'Home & Utilities',
                         'tags': ['internet']},
            'Gifts': {'category' : 'Gifts',
                      'tags': []},
            'Insurance – Property': {'category' : 'Home & Utilities',
                                     'tags': ['insurance']},
            'Books, Music, Movies & DVDs': {'category' : 'Leisure',
                                            'tags': ['music', 'books']},
            'Insurance – Health': {'category' : 'Health & Personal Care',
                                   'tags': ['insurance']},
            'Miscellaneous Expense': {'category' : 'Other',
                                      'tags': []},
            'Military taxe': {'category' : 'Taxes',
                              'tags': ['military tax']},
            'Parking': {'category' : 'Transport',
                        'tags': ['parking']},
            'Furniture & Electronics': {'category' : 'Home & Utilities',
                                        'tags': []},
            'Personal Net Worth': {'category' : 'Other',
                                   'tags': []},
            'Home Phone': {'category' : 'Home & Utilities',
                           'tags': ['phone']},
            'Subscriptions': {'category' : 'Leisure',
                              'tags': ['subscriptions']},
            'Donations': {'category' : 'Charity',
                          'tags': []},
            'Clothes': {'category' : 'Clothing & Footwear',
                        'tags': []},
            'Home Improvement & Maintenance': {'category' : 'Home & Utilities',
                                               'tags': ['home improvement']}
             }

In [ ]:
categories_income = {'Transfer Clearing': {'category' : 'Transfer',
                                           'tags': []},
                     'Restaurants, Coffee & Bars': {'category' : 'Reimbursements',
                                                    'tags': []},
                     'Accounts Payable': {'category' : 'Loans',
                                          'tags': ['payable']},
                     'Travel & Vacation': {'category' : 'Reimbursements',
                                           'tags': []},
                     'Accounts Receivable': {'category' : 'Loans',
                                             'tags': ['receivable']},
                     'Personal Net Worth': {'category' : 'Other',
                                            'tags': []},
                     'Entertainment': {'category' : 'Reimbursements',
                                       'tags': []},
                     'Rental Car & Taxi': {'category' : 'Reimbursements',
                                           'tags': []},
                     'Owner Investment / Drawings': {'category' : 'Loans',
                                                    'tags': ['owner investment']},
                     'Income & Bonuses': {'category' : 'Salary',
                                          'tags': []},
                     'Interest Income': {'category' : 'Other',
                                         'tags': ['interest']},
                     'Bank & Credit Card Fees': {'category' : 'Reimbursements',
                                                 'tags': []},
                     'Gain on Foreign Exchange': {'category' : 'Other',
                                                  'tags': []},
                     'Mortgage or Rent': {'category' : 'Reimbursements',
                                          'tags': []},
                     'Alcohol & Tobacco': {'category' : 'Reimbursements',
                                           'tags': []},
                     'Split': {'category' : 'Other',
                               'tags': []},
                     'Insurance – Health': {'category' : 'Reimbursements',
                                            'tags': []},
                     'Subscriptions': {'category' : 'Reimbursements',
                                       'tags': []}
                    }

In [ ]:
category_tag_type = 'expense'
category = 'Home & Utilities'
tag = 'home improvement'

print('The category "{}" has the ID {}.'.format(category, get_category_id(category, category_tag_type)))
print('The tag "{}" has the ID {}.'.format(tag, get_tag_id(tag, category_tag_type)))

In [ ]:
def convert_categories_to_id(categ_dict, categ_type):
    for category in categ_dict:
        categ_dict[category]['category'] = get_category_id(categ_dict[category]['category'], categ_type)
        if categ_dict[category]['tags']:
            for i in range(len(categ_dict[category]['tags'])):
                categ_dict[category]['tags'][i] = get_tag_id(categ_dict[category]['tags'][i], categ_type)

    return categ_dict

In [ ]:
categories = convert_categories_to_id(categories, 'expense')
categories

In [ ]:
categories_income = convert_categories_to_id(categories_income, 'income')
categories_income

In [ ]:
# Explore category
print(toshl_categories[toshl_categories['id'] == '53595248'].type.values)

# Explore tag
print(toshl_tags[toshl_tags['id'] == '53595248'].type.values)

In [ ]:
def check_missing_category(df):
    df_income = df[df['Amount (One column)'] > 0]
    df_expense = df[df['Amount (One column)'] < 0]
    df_income_split = df[(df['split'] != '') & (df['Amount (One column)'] > 0)]
    df_expense_split = df[(df['split'] != '') & (df['Amount (One column)'] < 0)]
    set_income = set()
    set_expense = set()

    # loop income categories from transactions
    for key in df_income['category'].value_counts().to_dict():
        if key not in categories_income:
            set_income.add(key)
    
    # loop expense categories from transactions
    for key in df_expense['category'].value_counts().to_dict():
        if key not in categories:
            set_expense.add(key)
    
    # loop income categories from split transactions
    print('\nIncome categories to map:\n')
    for i in df_income_split.index:
        for key in df.loc[i, 'split']:
            if key not in categories_income:
                set_income.add(key)
    for i in set_income:
        print(i)
    
    # loop expense categories from split transactions
    print('\nExpense categories to map:\n')
    for i in df_expense_split.index:
        for key in df.loc[i, 'split']:
            if key not in categories:
                set_expense.add(key)
    for i in set_expense:
        print(i)

In [ ]:
def upload_to_toshl(dataframe, account_id, currency_code):

    for i in dataframe.index:

        amount = dataframe.loc[i, 'Amount (One column)']
        currency = {'code' : currency_code}
        date = dataframe.loc[i, 'Transaction Date']
        desc = dataframe.loc[i, 'desc']
        
        # if it's an income
        if float(amount) > 0:
            category = categories_income[dataframe.loc[i, 'category']]
        else:
            category = categories[dataframe.loc[i, 'category']]

        data = {
            'amount': amount,
            'currency': currency,
            'date': date,
            'desc': desc,
            'account': account_id,
            'category': category['category']
        }
        
        
        # if tags existing, add it to the upload
        if category['tags']:
            data['tags'] = category['tags']
            
        # if split transaction
        if dataframe.loc[i, 'split'] != '':
            # post parent transaction
            parent = post_toshl('/entries', data)
            parent_id = re.findall(r'\d+', parent)[0]
            
            for key in dataframe.loc[i, 'split']:
                
                amount = dataframe.loc[i, 'split'][key]
                data['amount'] = amount
                
                # if it's an income
                if float(amount) > 0:
                    category = categories_income[key]
                else:
                    category = categories[key]
                    
                data['category'] = category['category']
                
                # if tags exists, add it to the upload
                if category['tags']:
                    data['tags'] = category['tags']
                else:
                    if 'tags' in data:
                        del data['tags']
                
                data['split'] = {'parent' : parent_id}
                
                post_toshl('/entries', data) 
                
        else:
            post_toshl('/entries', data) 

In [ ]:
def merge_data(df1, df2):
    
    df1['abs_transaction'] = 0
    
    for i in df1.index:
        df1.loc[i, 'abs_transaction'] = abs(float(df1.loc[i, 'Amount (One column)']))
    
    df2 = df2.astype({'account_value': float})
    
    new_df = pd.merge(df1, df2,  how='left', left_on=['Transaction Date','Transaction Description', 'abs_transaction'],
                                             right_on = ['transaction_date','transaction_description', 'account_value'])
    
    new_df = new_df.drop(['abs_transaction'], axis=1)
    
    print('After merging: {} rows and {} columns'.format(new_df.shape[0], new_df.shape[1]))
    
    return new_df

In [ ]:
def cleaning_merge_data(df):
    get_trans_id = lambda i : df.iloc[i].at['Transaction ID']
    get_trans_amt = lambda i : abs(df.iloc[i].at['Amount (One column)'])
    def count_transaction_id(index):
        return len(df[df['Transaction ID'] == get_trans_id(index)])

    i = 0
    while i < df.shape[0]-1:
        # if same transaction and same amount
        if get_trans_id(i) == get_trans_id(i+1) and get_trans_amt(i) == get_trans_amt(i+1):
            df = df.drop([df.iloc[i+1].name])
            i += 1
        else:
            i += 1

    print('After cleaning: {} rows and {} columns'.format(df.shape[0], df.shape[1]))
    
    return df

In [ ]:
def drop_clean_data(df):
    for i in df.index:
        # if income
        if float(df.loc[i, 'Amount (One column)']) > 0:
            df.loc[i, 'Amount (One column)'] = df.transaction_value.loc[i]
        # if expense
        else:
            df.loc[i, 'Amount (One column)'] = '-'  + str(df.transaction_value.loc[i])

        if df.loc[i, 'split'] != '':
            currency_rate = (float(df.transaction_value.loc[i]) / float(df.account_value.loc[i]))
            if currency_rate < 1:
                currency_rate = (float(df.account_value.loc[i]) / float(df.transaction_value.loc[i]))
            for key in df.loc[i, 'split']:
                df.loc[i, 'split'][key] = round(float(df.loc[i, 'split'][key]) * currency_rate, 2)

    df = df.drop(['Transaction Description',
                  'anchor_account_name',
                  'transaction_date',
                  'transaction_description',
                  'account_value',
                  'account_currency',
                  'transaction_currency',
                  'transaction_value'], axis=1)
    return df

In [ ]:
def check_size_data(df1, df2):
    df1.shape[0]
    df2.shape[0]
    
    if df1.shape[0] == df2.shape[0]:
        print('Success. Same size as {}'.format(df1.shape[0]))
    else:
        print('Error. Size of {} and {}.'.format(df1.shape[0], df2.shape[0]))

# To repeat for every account

In [ ]:
print(df['Account Name'].value_counts())

In [ ]:
df_to_upload = df.loc[df['Account Name'] == 'Cash on Hand - CHF']
df_to_upload

In [ ]:
check_missing_category(df_to_upload)

# If currency is not Euro

In [ ]:
web_wave_guid_account = '721580773001422957' # replace with guid of the wave web acccount
df_waveapps_web = get_wave_web_data(web_wave_guid_account)

In [ ]:
check_size_data(df_to_upload, df_waveapps_web)

### ---
### if not the same because sample too small:

In [ ]:
df_waveapps_web_2 = get_wave_web_data(web_wave_guid_account, order = 'ASC')

In [ ]:
new_df = pd.concat([df_waveapps_web, df_waveapps_web_2], ignore_index=True)

new_df = new_df.drop_duplicates(subset = 'transaction_guid')

In [ ]:
df_waveapps_web = new_df
check_size_data(df_to_upload, df_waveapps_web)

### ---

In [ ]:
df_waveapps_web = waveapps_web_cleaning(df_waveapps_web)

In [ ]:
new_df = merge_data(df_to_upload, df_waveapps_web)

In [ ]:
clean_new_df = cleaning_merge_data(new_df)

In [ ]:
df_to_upload = drop_clean_data(clean_new_df)

# Do the upload

In [ ]:
df_to_upload

In [ ]:
for i in toshl_accounts.index:
    print(f'Toshl {toshl_accounts.id.iloc[i]} : {toshl_accounts.name.iloc[i]}')

In [ ]:
df_to_push = df_to_upload
remote_account_id = 00000 # replace with the toshl account ID
currency_code = 'CHF'

In [ ]:
upload_to_toshl(df_to_push, remote_account_id, currency_code)

In [ ]:
df_toshl = pd.DataFrame(get_toshl(
    '/entries?from=2017-03-04&to=2023-03-07&accounts={}&per_page=400'.format(remote_account_id)))

number_transactions = df_to_push.shape[0]
for i in df_to_push[df_to_push['split'] != ''].index:
    for key in df_to_push[df_to_push['split'] != ''].loc[i, 'split']:
        number_transactions += 1
    number_transactions -= 1

if df_to_push.shape[0] == df_toshl.shape[0]:
    print('Success. Same number of rows. No split transactions.')
elif number_transactions == df_toshl.shape[0]:
    print('Success. Same number of rows with split transactions. {} transactions.'.format(number_transactions))
elif df_toshl.shape[0] == 0:
    print('Remote account empty.')
else: print('Error. Number of row do not match.')
    
df_toshl.head()

In [ ]:
# df_toshl.to_csv('backup-toshl.csv')

# If needed, empty the remote account

In [ ]:
delete_all_toshl(remote_account_id)